# LinkedIn Scraper

Данная работа адресует проблему автоматизированного поиска данных профилей пользователей LinkedIn.

Иерархия связей профилей в LinkedIn предусматривает невозможность просмотра профилей пользователей, находящихся вне вашей сети (3+ уровень "connection"). Во встроенном поиске профили таких людей отображаются просто как LinkedIn Member. Соответсвенно, если Ваш профиль не имеет "connections" - связей, Вы не сможете просматривать в поиске практически никакие профили.

Однако публичные (открытые) профили LinkedIn появляются в поисковой выдаче Google, несмотря на то, что во внутреннем поиске LinkedIn получить к ним доступ невозможно.

Тогда возникла идея реализовать автоматизацию получения данных профилей пользователей по следующему алгоритму:
1. Получение списка ссылок на профили LinkedIn
    - Использование Google Custom Search Engine и продвинутых средств поиска. Поиск inurl:linkedin.com/in/ + тэги intitle и др.
    - Scraping по поисковой выдаче
    - Формирование списка ссылок в .csv
<br><br>
2. Авторизация в LinkedIn в webdriver
    - Использование учетных данных аккаунта
    - Сохранение cookies
<br><br>
3. Скрэпинг профилей из списка ссылок
    - Подключение cookies (для сохранения сессии)
    - Сохранение фото (скриншот img элемента в браузере) в файл изображения (адрес_профиля.png) и описания профиля About (при наличии) в текстовый файл (адрес_профиля.txt)
    - Запись данных в meta.csv (имя фамилия, должность, ссылка на профиль, пути к файлам, timestamp получения данных)
    
<br>|-- <i>link_list.csv</i> - список ссылок, полученных после скрапинга поисковой выдачи
<br>|-- <i>meta_scraped.csv</i> - список обработанных профилей linkedin (с ошибкой в поле about_path)
<br>|-- <i>meta_corrected.csv</i> - список обработанных профилей linkedin (ошибка исправлена)
<br>|-- <i>linkedin_scraper.ipynb</i> - файл блокнота
<br>|-- <b><i>output/</i></b>
<br>|   |-- <b><i>img</i></b> - полученные скриншоты фото профилей пользователей LinkedIn
<br>|   |-- <b><i>txt</i></b> - полученный текст из раздела About профилей пользователей LinkedIn

In [ ]:
from dotenv import load_dotenv
import os

Для упрощения задачи скрэпинга используется Google Programmable Search Engine:
    Создается поиск с параметром поиска по сайту linkedin.com
    https://programmablesearchengine.google.com/about/
    Используем ENGINE_ID созданного поиска

В файле .env необходимо указать данные аккаунта LinkedIn и id поискового движка.
Мой engine_id: d776545fc25f64d13

In [ ]:
load_dotenv()

LINKEDIN_LOGIN = os.environ['LINKEDIN_LOGIN']
LINKEDIN_PASSWORD = os.environ['LINKEDIN_PASSWORD']
ENGINE_ID = os.environ['ENGINE_ID']

Поисковая выдача Programmable Search Engine ограничена 10 страницами (~100 результатов). Поэтому для достижения требуемого количества ссылок (200) используется несколько поисковых запросов.

In [ ]:
JOB_TITLE = ['Software Developer', 'Software Developer', 'Software Developer', 'Software Developer']
COMPANY = ['Apple', 'Microsoft', 'IBM', 'Intel']
LOCATION = ['Cupertino', 'Redmond', 'New York', 'California']

Получение списка ссылок из выдачи, браузер автоматически переходит по страницам, скрипт ждет ввода от пользователя при появлении капчи (необходимо решить капчу и нажать Enter в поле для ввода ниже ячейки для продолжения работы скрипта)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def handle_captcha():
    while True:
        try:
            captcha = driver.find_element(By.CLASS_NAME, 'gs-captcha-msg')
            if captcha.is_displayed():
                print('Please solve the captcha')
                _ = input("Press Enter to continue: ")
            else:
                break
        except:
            break

def go_next_page():
    current_page = driver.find_element(By.CLASS_NAME, 'gsc-cursor-current-page')
    next_page = driver.find_element(By.XPATH, f"//div[@aria-label='Page {int(current_page.text) + 1}']")
    next_page.click()

def get_linkedin_profile_links():
    titles = driver.find_elements(By.CSS_SELECTOR, "a.gs-title")
    for title in titles:
        url = title.get_attribute('data-ctorig')
        data.append(url)

data = []

driver = webdriver.Chrome()

for i in range(len(JOB_TITLE)):
    custom_search_engine_url = f'https://cse.google.com/cse?cx={ENGINE_ID}'

    driver.get(custom_search_engine_url)
    time.sleep(3)

    search_input = driver.find_element(By.TAG_NAME, 'input')
    search_input.send_keys(f'inurl:linkedin.com/in/ intitle:{JOB_TITLE[i]} intitle:{COMPANY[i]} {LOCATION[i]}')
    search_input.send_keys(Keys.ENTER)
    time.sleep(3)

    pages = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
    for _ in range(len(pages)-1):
        time.sleep(3)
        try:
            handle_captcha()
        except:
            pass

        get_linkedin_profile_links()
        try:
            go_next_page()
        except:
            break

driver.quit()


Please solve the captcha
Press Enter to continue: 


В списке присутствуют дубликаты (из-за нестинга элементов, возможно, можно подобрать css селектор получше). Поэтому используем еще предобработку

In [ ]:
import pandas as pd

data_df = pd.DataFrame(data)
data_df.drop_duplicates(inplace=True)
data_df.dropna(inplace=True)
data_df.reset_index(inplace=True)
data_df.drop('index', axis=1, inplace = True)

Сохраняем в csv (можно было сделать сразу запись в файл, но решил оставить так)

In [ ]:
data_df.to_csv('link_list.csv')

In [ ]:
import pandas as pd

data_df = pd.read_csv('link_list.csv')

In [ ]:
data_df

,Unnamed: 0,0
0,0,https://www.linkedin.com/in/tylerjameswallace
1,1,https://www.linkedin.com/in/max-waldor
2,2,https://www.linkedin.com/in/joelyoung2
3,3,https://www.linkedin.com/in/devjmendoza
4,4,https://ng.linkedin.com/in/orlando-lorenzo-285...
...,...,...
240,240,https://mx.linkedin.com/in/yram/en
241,241,https://www.linkedin.com/in/susan-stiles-56387511
242,242,https://www.linkedin.com/in/james-cauble
243,243,https://www.linkedin.com/in/jayda-nance


In [ ]:
# для второго запуска (после auth wall)
# data_df = data_df.loc[125:, :]

Авторизация с данными нашего аккаунта + запись cookies

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import datetime


import json

driver = webdriver.Chrome()

driver.get("https://linkedin.com/uas/login")

time.sleep(5)

username = driver.find_element(By.ID, "username")
username.send_keys(LINKEDIN_LOGIN)

pword = driver.find_element(By.ID, "password")
pword.send_keys(LINKEDIN_PASSWORD)

driver.find_element(By.XPATH, "//button[@type='submit']").click()

cookies = driver.get_cookies()

with open('cookies.json', 'w') as file:
    json.dump(cookies, file)

driver.quit()

Итерирование по списку полученных ссылок

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import datetime
import json

from csv import writer

from pathlib import Path


driver = webdriver.Chrome()

with open('cookies.json', 'r') as file:
    cookies = json.load(file)

driver.get('https://linkedin.com')

for cookie in cookies:
    driver.add_cookie(cookie)


my_file = Path("meta_scraped.csv")
if not my_file.is_file():
    with open(f'meta_scraped.csv', 'w') as file:
        writer_object = writer(file)
        writer_object.writerow(['name', 'job_title', 'location', 'linkedin_url', 'image_path', 'about_path', 'retrieved_at'])
        file.close()

for link in data_df.values:
    try:
        driver.get(link[1])
        time.sleep(5)
        name = driver.find_element(By.CLASS_NAME, 'text-heading-xlarge').text
        try:
            login = link[1].split('https://www.linkedin.com/in/')[1]
        except:
            login = name.lower()
        try:
            job_title = driver.find_element(By.CLASS_NAME, 'text-body-medium').text
        except:
            job_title = 'unknown'
        try:
            location = driver.find_element(By.CLASS_NAME, 'text-body-small').text
        except:
            location = 'unknown'
        img = driver.find_element(By.CLASS_NAME, 'pv-top-card-profile-picture__image')
        image_path = f'output/img/{login}.png'
        with open(image_path, 'wb') as file:
            file.write(img.screenshot_as_png)
            file.close()
        try:
            about_text = driver.find_element(By.XPATH, '//*[@id="profile-content"]/div/div[2]/div/div/main/section[2]/div[3]/div/div/div/span[1]')
            about_path = f'output/txt/{login}.txt'
            with open(about_path, 'w') as file:
                file.write(about_text.text)
                file.close()
        except:
            with open(f'output/txt/{login}.txt', 'w') as file:
                file.write('')
                file.close()

        retrieved_at = datetime.datetime.now()

        row = [name, job_title, location, link[1], image_path, about_path, retrieved_at]

        with open('meta_scraped.csv', 'a') as file:
            writer_object = writer(file)
            writer_object.writerow(row)
            file.close()
    except:
        print(f'Link {link[1]} could not be parsed')
        continue



После истечения времени сессии linkedin переадресует на authwall. Без обработки authwall удалось получить 122 записи. Появляется предупреждение о запрете использования средств автоматизации.

Возможное решение: использование нескольких аккаунтов (+ куки + впн страны регистрации аккаунта) - при появлении authwall закрытие сессии, рестарт браузера, загрузка куки, подмена user-agent.

Для получения 200 записей достаточно одного аккаунта (протестировано на личном аккаунте, регистрация 2021 г.). Следующий запуск произведен на следующий день после появления authwall.

При обработке 78 ссылок authwall не появился

! Поместил объявление about_path в try:, что привело к неправильной записи about_path в метаданные. Пути к изображениям верные

! Сами текстовые файлы сохранились, можно изменить файл meta.csv

In [ ]:
meta = pd.read_csv('meta_scraped.csv')

In [ ]:
meta

,name,job_title,location,linkedin_url,image_path,about_path,retrieved_at
0,Tyler Wallace,Software Engineer at Apple,Apple,https://www.linkedin.com/in/tylerjameswallace,output/img/tylerjameswallace.png,output/txt/tylerjameswallace.txt,2024-03-01 19:15:39.754356
1,Max Waldor,Software Developer specializing in computer gr...,Apple,https://www.linkedin.com/in/max-waldor,output/img/max-waldor.png,output/txt/max-waldor.txt,2024-03-01 19:16:41.280209
2,Joel Young,Software Engineer at Apple Maps,Apple,https://www.linkedin.com/in/joelyoung2,output/img/joelyoung2.png,output/txt/max-waldor.txt,2024-03-01 19:17:53.974292
3,Jose Mendoza,GPU Software Engineer at Apple,Apple,https://www.linkedin.com/in/devjmendoza,output/img/devjmendoza.png,output/txt/devjmendoza.txt,2024-03-01 19:19:22.651591
4,Orlando Lorenzo,--,Apple,https://ng.linkedin.com/in/orlando-lorenzo-285...,output/img/orlando lorenzo.png,output/txt/devjmendoza.txt,2024-03-01 19:20:07.169459
...,...,...,...,...,...,...,...
195,Janet Hobbins,Managing Consultant at IBM,IBM,https://www.linkedin.com/in/janethobbins,output/img/janethobbins.png,output/txt/janisjlee.txt,2024-03-02 16:41:50.868980
196,Ciara J.,PM & Market Researcher @ IBM | Market Research...,IBM,https://www.linkedin.com/in/ciara-j-b92bb711b,output/img/ciara-j-b92bb711b.png,output/txt/janisjlee.txt,2024-03-02 16:42:15.478774
197,Brooke Nugent,Information Technology Consultant - Project Ma...,IBM,https://www.linkedin.com/in/brooke-nugent-a09b5aa,output/img/brooke-nugent-a09b5aa.png,output/txt/janisjlee.txt,2024-03-02 16:42:50.290659
198,Vincent Quartararo,"Data Center SME, Project Manager, Media Destru...",IBM,https://www.linkedin.com/in/vincent-quartararo...,output/img/vincent-quartararo-38658465.png,output/txt/janisjlee.txt,2024-03-02 16:43:12.384494


Теперь правильно:

In [ ]:
def correct_about_text(row):
    row = row.replace('/img/', '/txt/')
    row = row.replace('.png', '.txt')
    return row

meta_corrected = meta.copy()

meta_corrected['about_path'] = meta_corrected['image_path']
meta_corrected['about_path'] = meta_corrected['about_path'].apply(correct_about_text)

In [ ]:
meta_corrected

,name,job_title,location,linkedin_url,image_path,about_path,retrieved_at
0,Tyler Wallace,Software Engineer at Apple,Apple,https://www.linkedin.com/in/tylerjameswallace,output/img/tylerjameswallace.png,output/txt/tylerjameswallace.txt,2024-03-01 19:15:39.754356
1,Max Waldor,Software Developer specializing in computer gr...,Apple,https://www.linkedin.com/in/max-waldor,output/img/max-waldor.png,output/txt/max-waldor.txt,2024-03-01 19:16:41.280209
2,Joel Young,Software Engineer at Apple Maps,Apple,https://www.linkedin.com/in/joelyoung2,output/img/joelyoung2.png,output/txt/joelyoung2.txt,2024-03-01 19:17:53.974292
3,Jose Mendoza,GPU Software Engineer at Apple,Apple,https://www.linkedin.com/in/devjmendoza,output/img/devjmendoza.png,output/txt/devjmendoza.txt,2024-03-01 19:19:22.651591
4,Orlando Lorenzo,--,Apple,https://ng.linkedin.com/in/orlando-lorenzo-285...,output/img/orlando lorenzo.png,output/txt/orlando lorenzo.txt,2024-03-01 19:20:07.169459
...,...,...,...,...,...,...,...
195,Janet Hobbins,Managing Consultant at IBM,IBM,https://www.linkedin.com/in/janethobbins,output/img/janethobbins.png,output/txt/janethobbins.txt,2024-03-02 16:41:50.868980
196,Ciara J.,PM & Market Researcher @ IBM | Market Research...,IBM,https://www.linkedin.com/in/ciara-j-b92bb711b,output/img/ciara-j-b92bb711b.png,output/txt/ciara-j-b92bb711b.txt,2024-03-02 16:42:15.478774
197,Brooke Nugent,Information Technology Consultant - Project Ma...,IBM,https://www.linkedin.com/in/brooke-nugent-a09b5aa,output/img/brooke-nugent-a09b5aa.png,output/txt/brooke-nugent-a09b5aa.txt,2024-03-02 16:42:50.290659
198,Vincent Quartararo,"Data Center SME, Project Manager, Media Destru...",IBM,https://www.linkedin.com/in/vincent-quartararo...,output/img/vincent-quartararo-38658465.png,output/txt/vincent-quartararo-38658465.txt,2024-03-02 16:43:12.384494


In [ ]:
meta_corrected.to_csv('meta_corrected.csv')